# FOOD AND BEVERAGE BUSINESS OPPORTUNITY SEEKER

This is an independent project centered around data science application in food and beverage industry. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 4>

1. <a href="#item1">Introduction</a>
    
2. <a href="#item2">Data</a>

</font>
</div>

## 1. Introduction

There is a businessman lives in the capital city of Indonesia, Jakarta. He wants to start a food and Beverage (FnB) business in the city where he is living, but he doesn't know how to start. Without any experience in this kind of business, he comes to us and tells us about his plan. This new businessman has two critical questions here, they are:
1. What kind of food and beverage business that he should start with?
2. Where in the city should he start this business?

He hopes that we can help him to answer these questions to lead him make a right decision. As a Data Scientist, you are challenged to support him by giving him best recommendation in order to build a sucessful FnB business.

## 2. Data

In order to answer those two questions, as a data scientist, we need to have data related to the food and beverage market in Jakarta. But first, we need to have list of neighborhoods in Jakarta to find the list of existing foods venues near each neighborhood.

In Indonesia, we name neighborhood as "Kecamatan". We will use list of neighborhood in the city of Jakarta from wikipedia page (<a href="https://id.wikipedia.org/wiki/Daftar_kecamatan_dan_kelurahan_di_Daerah_Khusus_Ibukota_Jakarta">here</a>). The coordinate of the neighborhood will be used to find nearest food venues. Foursquare has public API that can provide us the data.  Foursquare has some account tiers for developers. Each tier has different set of available features. You can read the full documentation <a href="https://developer.foursquare.com/comparison">here</a> and <a href="https://developer.foursquare.com/docs">here</a>. In this project we are using personal tier. You can see the available endpoint for this tier <a href="https://developer.foursquare.com/docs/api/endpoints">here</a>. In this tier, we can get the data of venues near the specific location with specific category. We can use this endpoint to find existing restaurant venues in Jakarta. The data should contain:
+ longitude
+ latitude
+ venue category

We will do some analysis on this data to answer the two questions. We can use the data in determining what kind of FnB business that we want to begin with and in which neighborhood we should start our new business.

## 3. Methodology

There are steps that we need to do to answer the business problem. Here they are:
1. Web scrapping Wikipedia Page to get list of neighborhood in Jakarta
2. Get longitude and latitude of each neighborhood
3. Get list of food venues near each neighborhood
4. Analyze these data using descriptive statistics
5. Cluster all these food venues using k-Mean algorithm
6. Analyze each cluster using bar chart of top nearby venue types vs frequency
7. Visualize the clusters in map

### 4. Let's Do This

Let's import all necessary library

In [3]:
# Basic Library
import pandas as pd
import numpy as np

# Library for http request operation and Web scrapping
import requests
from bs4 import BeautifulSoup

# library for geocoding
from geopy.geocoders import Nominatim
import time

# Library for visualization
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# Library for k-means clustering
from sklearn.cluster import k_means

#### 4.1. Web Scrapping

We will retrieve list of neighborhood in Jakarta from wikipedia page <a href="https://id.wikipedia.org/wiki/Daftar_kecamatan_dan_kelurahan_di_Daerah_Khusus_Ibukota_Jakarta">here</a> using beautifulsoup module. 

In [38]:
# URL of the wikipedia page
url = "https://id.wikipedia.org/wiki/Daftar_kecamatan_dan_kelurahan_di_Daerah_Khusus_Ibukota_Jakarta"
source = requests.get(url).text

# Create beautiful soup object
soup = BeautifulSoup(source, 'lxml')

# Find table that contains the neighborhoods data in the page
tables = soup.findAll("table", {"class": "wikitable sortable"})
jakarta_regions = ["Jakarta Pusat", "Jakarta Utara", "Jakarta Timur", "Jakarta Selatan", "Jakarta Barat"]

neighborhood_df = pd.DataFrame({"Region":[], "Neighborhood":[]})
for i,table in enumerate(tables[1:-1]):
    body = table.find('tbody')
    region = jakarta_regions[i]
    for row in body.find_all('tr')[1:-1]:
        items = row.find_all('td')
        neighborhood_name = items[1].a.text
        new_row = {"Region":region, "Neighborhood":neighborhood_name}
        neighborhood_df = neighborhood_df.append(new_row, ignore_index=True)
        
neighborhood_df.head(10)

,Region,Neighborhood
0,Jakarta Pusat,Cempaka Putih
1,Jakarta Pusat,Cempaka Putih
2,Jakarta Pusat,Johar Baru
3,Jakarta Pusat,Kemayoran
4,Jakarta Pusat,Menteng
5,Jakarta Pusat,Sawah Besar
6,Jakarta Pusat,Senen
7,Jakarta Pusat,Tanah Abang
8,Jakarta Utara,Cilincing
9,Jakarta Utara,Kelapa Gading


By some prior knowledge about the city neighborhood, I can see that there is an incorrect neighborhood name in the table. This mistake is already there since in the wikipedia page. Two neighborhoods have the same name, but one of them is actually has a different name. Using my prior knowledge, I set one of them to the actual value, "Gambir". Yes, one missing neighborhood in Jakarta Pusat is Gambir. So let's replace one of them with Gambir.

In [42]:
neighborhood_df.set_value(1, "Neighborhood", "Gambir")
neighborhood_df.head()

C:\Users\Siahaan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """Entry point for launching an IPython kernel.


,Region,Neighborhood
0,Jakarta Pusat,Cempaka Putih
1,Jakarta Pusat,Gambir
2,Jakarta Pusat,Johar Baru
3,Jakarta Pusat,Kemayoran
4,Jakarta Pusat,Menteng


#### 4.2. Get longitude and latitude of each neighborhood

In this section, we will try to get the coordinate of each neighborhood. We are using Nominatim API from geopy geocoding module for this. This API is an open source API and based on OpenStreetMap, it is not odd if the data of longitude and latitude available in this API is not complete. Let's try this.

In [43]:
# Function to get longitude and latitude of each neighborhood in neighborhood dataframe
def getLongLat(neigh_df):
    # Create Nominatim geolocator agent object
    geolocator = Nominatim(user_agent="test")
    
    # new dataframe of longitude and latitude data
    longlat_df = pd.DataFrame({"Neighborhood":[], "Longitude":[], "Latitude":[]})
    
    #iterate through each row
    for index, row in neigh_df.iterrows():
        neighborhood = row["Neighborhood"]
        region = row["Region"]
        location = geolocator.geocode(neighborhood + "," + region, timeout=5)
        new_row = {
            "Neighborhood": neighborhood,
            "Longitude": location.longitude,
            "Latitude": location.latitude,
        }
        
        longlat_df = longlat_df.append(new_row, ignore_index=True)
        time.sleep(1)
    print(neigh_df)
    print(longlat_df)
    neigh_df.set_index("Neighborhood", inplace=True)
    longlat_df.set_index("Neighborhood", inplace=True)
    result_df = pd.concat([neigh_df, longlat_df], axis=1)
    
    return result_df

In [44]:
neighborhood_df = getLongLat(neighborhood_df)

             Region       Neighborhood
0     Jakarta Pusat      Cempaka Putih
1     Jakarta Pusat             Gambir
2     Jakarta Pusat         Johar Baru
3     Jakarta Pusat          Kemayoran
4     Jakarta Pusat            Menteng
5     Jakarta Pusat        Sawah Besar
6     Jakarta Pusat              Senen
7     Jakarta Pusat        Tanah Abang
8     Jakarta Utara          Cilincing
9     Jakarta Utara      Kelapa Gading
10    Jakarta Utara               Koja
11    Jakarta Utara         Pademangan
12    Jakarta Utara        Penjaringan
13    Jakarta Utara      Tanjung Priok
14    Jakarta Timur             Cakung
15    Jakarta Timur           Cipayung
16    Jakarta Timur            Ciracas
17    Jakarta Timur        Duren Sawit
18    Jakarta Timur         Jatinegara
19    Jakarta Timur        Kramat Jati
20    Jakarta Timur            Makasar
21    Jakarta Timur           Matraman
22    Jakarta Timur         Pasar Rebo
23    Jakarta Timur        Pulo Gadung
24  Jakarta Selatan      

In [24]:
neighborhood_df.head()

,Region
Neighborhood,
Cempaka Putih,Jakarta Pusat
Cempaka Putih,Jakarta Pusat
Johar Baru,Jakarta Pusat
Kemayoran,Jakarta Pusat
Menteng,Jakarta Pusat


After chec 

In [61]:
neighborhood_df.to_csv(r"jakarta_neighborhoods.csv")

In [69]:
neighborhood_df = pd.read_csv("jakarta_neighborhoods_with_coordinates.csv", delimiter=";")
neighborhood_df.head()

,Neighborhood,Region,Longitude,Latitude
0,Cempaka Putih,Jakarta Pusat,106.8680,-6.1827
1,Gambir,Jakarta Pusat,106.8148,-6.1703
2,Johar Baru,Jakarta Pusat,106.8577,-6.1868
3,Kemayoran,Jakarta Pusat,106.8473,-6.1604
4,Menteng,Jakarta Pusat,106.8326,-6.1940


In [70]:
neighborhood_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
Neighborhood    42 non-null object
Region          42 non-null object
Longitude       42 non-null float64
Latitude        42 non-null float64
dtypes: float64(2), object(2)
memory usage: 1.4+ KB


In [72]:
CLIENT_ID = 'VQG3LESIQX2EBRDST02BYBPDNHRXFVVVUHD3S1WEWWISTOPG' # your Foursquare ID
CLIENT_SECRET = 'BLIANNOYZTV55LD5TV3L5SP1NITYZJVYXD3NSIX505C1IASJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ', CLIENT_ID)
print('CLIENT_SECRET:', CLIENT_SECRET)

Your credentails:
CLIENT_ID:  VQG3LESIQX2EBRDST02BYBPDNHRXFVVVUHD3S1WEWWISTOPG
CLIENT_SECRET: BLIANNOYZTV55LD5TV3L5SP1NITYZJVYXD3NSIX505C1IASJ


In [67]:
def getFoodVenues(neighborhood_df, radius=2000):
    FOOD_CATEGORY_ID = "4d4b7105d754a06374d81259"
    LIMIT = 50
    venues_list=[]
    
    i = 1
#     for region, lat, lng in zip(regions, latitudes, longitudes): 
    for index,row in neighborhood_df.iterrows():
        region = row["Region"]
        neighborhood = row["Neighborhood"]
        longitude = row["Longitude"]
        latitude = row["Latitude"]
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT,
            FOOD_CATEGORY_ID)
        
#         print(url)
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except Exception as e:
            print("GAGAL")
            #print(results)
            
        # return only relevant information for each nearby venue
        venues_list.append([(
            neighborhood,
            region,
            latitude, 
            longitude, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Region',
                  'Latitude', 
                  'Longitude',
                  'Venue_ID',
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
        
    return(nearby_venues)

In [76]:
jakarta_food_venues = getFoodVenues(neighborhood_df)

In [77]:
jakarta_food_venues.head()

,Neighborhood,Region,Latitude,Longitude,Venue_ID,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Cempaka Putih,Jakarta Pusat,-6.1827,106.868,4bf0d4da1f17ef3bb1d89bc8,Pagi Sore,-6.176376,106.871474,Padangnese Restaurant
1,Cempaka Putih,Jakarta Pusat,-6.1827,106.868,558b4c3d498e521f198a3a5e,DIMSUM-ING,-6.176087,106.872188,Dim Sum Restaurant
2,Cempaka Putih,Jakarta Pusat,-6.1827,106.868,5371d62f498ea16a1fa978e2,Sentral Al-Jazeerah Restaurant & Cafe,-6.192718,106.863584,Moroccan Restaurant
3,Cempaka Putih,Jakarta Pusat,-6.1827,106.868,59392cbd780eee50d259ee29,Mujigae Bibimbap & Casual Korean Food,-6.189437,106.874310,Korean Restaurant
4,Cempaka Putih,Jakarta Pusat,-6.1827,106.868,54a3fd0d498ee74ea1557581,Warna Bistro & Coffee,-6.176167,106.864428,Bistro


In [79]:
jakarta_food_venues.shape

(1988, 9)

In [81]:
# one hot encoding
jakarta_onehot = pd.get_dummies(jakarta_food_venues[['Venue_Category']], prefix="", prefix_sep="")

columns = jakarta_onehot.columns

# add neighborhood column back to dataframe
jakarta_onehot['Neighborhood'] = jakarta_food_venues['Neighborhood'] 
jakarta_onehot['Region'] = jakarta_food_venues['Region']

# move neighborhood column to the first column
fixed_columns = ["Neighborhood", "Region"] + list(columns)
jakarta_onehot = jakarta_onehot[fixed_columns]

jakarta_onehot.head()

,Neighborhood,Region,Acehnese Restaurant,African Restaurant,American Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Steakhouse,Sundanese Restaurant,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Cempaka Putih,Jakarta Pusat,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Cempaka Putih,Jakarta Pusat,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cempaka Putih,Jakarta Pusat,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cempaka Putih,Jakarta Pusat,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cempaka Putih,Jakarta Pusat,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
jakarta_grouped = jakarta_onehot.groupby("Neighborhood").mean().reset_index()
jakarta_grouped.head()

,Neighborhood,Acehnese Restaurant,African Restaurant,American Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Balinese Restaurant,...,Steakhouse,Sundanese Restaurant,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Cakung,0.00,0.0,0.054054,0.081081,0.0,0.027027,0.00,0.081081,0.0,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Cempaka Putih,0.02,0.0,0.000000,0.020000,0.0,0.040000,0.00,0.040000,0.0,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Cengkareng,0.00,0.0,0.000000,0.120000,0.0,0.000000,0.02,0.040000,0.0,...,0.02,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cilandak,0.00,0.0,0.000000,0.080000,0.0,0.000000,0.00,0.060000,0.0,...,0.06,0.02,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Cilincing,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
jakarta_grouped.shape

(42, 80)

In [84]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = jakarta_grouped['Neighborhood']

for ind in np.arange(jakarta_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(jakarta_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cakung,Indonesian Restaurant,Fast Food Restaurant,Asian Restaurant,Bakery,Food Court,Japanese Restaurant,American Restaurant,Noodle House,Food Truck,Pizza Place
1,Cempaka Putih,Indonesian Restaurant,Pizza Place,Café,Restaurant,Seafood Restaurant,Food Truck,Fast Food Restaurant,Indonesian Meatball Place,BBQ Joint,Japanese Restaurant
2,Cengkareng,Asian Restaurant,Indonesian Restaurant,Noodle House,Café,Chinese Restaurant,Pizza Place,Food Court,Fried Chicken Joint,Food Truck,Fast Food Restaurant
3,Cilandak,Indonesian Restaurant,Café,Food Truck,French Restaurant,Asian Restaurant,Steakhouse,Bakery,Padangnese Restaurant,Restaurant,Fast Food Restaurant
4,Cilincing,Deli / Bodega,Indonesian Restaurant,Seafood Restaurant,Diner,Wings Joint,Fish & Chips Shop,Dim Sum Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


In [93]:
# set number of clusters
kclusters = 5

jakarta_grouped_clustering = jakarta_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = k_means(X=jakarta_grouped_clustering, n_clusters=kclusters, random_state=0)

# check cluster labels generated for each row in the dataframe
len(kmeans[1])

42

In [94]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans[1])

jakarta_merged = neighborhood_df

# merge jakarta_grouped with jakarta_neighborhood_df to add latitude/longitude for each neighborhood
jakarta_merged = jakarta_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [95]:
jakarta_merged.head() # check the last columns!

,Neighborhood,Region,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cempaka Putih,Jakarta Pusat,106.8680,-6.1827,2,Indonesian Restaurant,Pizza Place,Café,Restaurant,Seafood Restaurant,Food Truck,Fast Food Restaurant,Indonesian Meatball Place,BBQ Joint,Japanese Restaurant
1,Gambir,Jakarta Pusat,106.8148,-6.1703,0,Indonesian Restaurant,Seafood Restaurant,Chinese Restaurant,Noodle House,Asian Restaurant,Soup Place,Restaurant,Padangnese Restaurant,Halal Restaurant,Fast Food Restaurant
2,Johar Baru,Jakarta Pusat,106.8577,-6.1868,1,Indonesian Restaurant,Restaurant,Seafood Restaurant,Fast Food Restaurant,Asian Restaurant,Chinese Restaurant,Pizza Place,Café,Soup Place,Padangnese Restaurant
3,Kemayoran,Jakarta Pusat,106.8473,-6.1604,0,Chinese Restaurant,Noodle House,Indonesian Restaurant,Food Truck,Food Court,Asian Restaurant,Diner,Restaurant,Indonesian Meatball Place,Dumpling Restaurant
4,Menteng,Jakarta Pusat,106.8326,-6.1940,2,Indonesian Restaurant,Restaurant,Japanese Restaurant,Buffet,Steakhouse,Chinese Restaurant,Thai Restaurant,Café,Breakfast Spot,Manadonese Restaurant


In [96]:
# create map
map_clusters = folium.Map(location=[-6.2088, 106.8456], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(jakarta_merged['Latitude'], jakarta_merged['Longitude'], jakarta_merged['Neighborhood'], jakarta_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

